In [75]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\vmakh\OneDrive\Desktop\Python-20220419T075236Z-001\Python\Data science for business\Lec 6\Lection6-20220325T130334Z-001\mammographic_masses.data", header=None, na_values='?')
df.head(5)

,0,1,2,3,4,5
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [76]:
print(df.shape, df.isna().sum(), sep='\n')

(961, 6)
0     2
1     5
2    31
3    48
4    76
5     0
dtype: int64


In [77]:
df=df.dropna(axis=0)
print(df.shape, df.isna().sum(), sep='\n')

(830, 6)
0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64


In [78]:
df.index = range(830)

In [79]:
df.iloc[:, -1].value_counts()

0    427
1    403
Name: 5, dtype: int64

In [80]:
from sklearn.model_selection import train_test_split

x_data = df.iloc[:,:-1]
y_data = df.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [81]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [82]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 76.07%
roc: 76.51%
recall: 74.70%
precision: 77.50%


In [83]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 101/403 as positives and unlabeling the rest


In [84]:
pos_sample

array([439, 188, 394, 151, 206, 684,  76, 713,  54, 300, 170, 193, 182,
       231, 137, 319, 623, 154, 179, 631, 438, 544, 465, 416, 567,  20,
       360,  52, 574, 760, 524, 403, 342, 177, 447, 102, 761, 198, 262,
       757, 453, 548, 497, 317, 211, 826, 344,  13, 183, 164, 727,  81,
       697, 278, 665, 719, 414, 485, 646, 526, 287, 607, 777,   1, 517,
       758, 331, 530, 489, 552, 340, 291, 462,  42, 576, 590, 585, 692,
       813, 297, 537, 454, 551, 636,  69, 392,  24,  87, 698, 161, 104,
       105, 150, 673, 285, 501, 729, 255, 232, 767, 564], dtype=int64)

In [85]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    729
 1    101
Name: class_test, dtype: int64


In [86]:
mod_data.head(10)

,0,1,2,3,4,5,class_test
0,5.0,67.0,3.0,5.0,3.0,1,-1
1,5.0,58.0,4.0,5.0,3.0,1,1
2,4.0,28.0,1.0,1.0,3.0,0,-1
3,5.0,57.0,1.0,5.0,3.0,1,-1
4,5.0,76.0,1.0,4.0,3.0,1,-1
5,3.0,42.0,2.0,1.0,3.0,1,-1
6,4.0,36.0,3.0,1.0,2.0,0,-1
7,4.0,60.0,2.0,1.0,2.0,0,-1
8,4.0,54.0,1.0,1.0,3.0,0,-1
9,3.0,52.0,3.0,4.0,3.0,0,-1


In [87]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [88]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(101, 7) (101, 7)


In [89]:
model_2 = xgb.XGBClassifier()

model_2.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict_2 = model_2.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict_2)

Classification results:
f1: 75.78%
roc: 78.04%
recall: 87.12%
precision: 67.06%


Если честно так и не понял зачем нужны были эти манипуляции с -1 и 1. Большого выиграша не наблюдается
